<a href="https://colab.research.google.com/github/aksh98/Collaborative_Filtering/blob/master/CFMid_Sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab-**commands**

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!ls
!google-drive-ocamlfuse drive

adc.json  drive  sample_data


In [0]:
%cd drive/Colab Notebooks/
%cd Assignments_sem7
!ls

/content/drive/Colab Notebooks
/content/drive/Colab Notebooks/Assignments_sem7
Assignment1_CF	      Assignment3.ipynb     IP.ipynb	      ml-100k
Assignment2_IA.ipynb  IA_Assignment1.ipynb  Mid_Sem_CF.ipynb


In [0]:
import pandas as pandas
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from numpy import newaxis
from sklearn.metrics import mean_absolute_error


In [0]:
header = ['user','item','rating','time']


# 100k ratings - 943 users - 1682 items
# each user rated atleast 20 movies
# user_id | item_id | rating | timestamp |

dataframe = pandas.read_csv('ml-100k/u.data',sep='\t',names=header)


In [0]:
dataframe.shape

(100000, 4)

In [0]:
dataframe.head()

,user,item,rating,time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
dataframe = dataframe.drop(labels='time',axis=1)

In [0]:
dataframe.head()

,user,item,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [0]:
Num_users = 943
Num_mov = 1682

In [0]:
# useritems = dataframe.pivot(index = 'user', columns ='item', values = 'rating').fillna(0)

# useritems.head()
# useritems.shape

def div0( arr1,arr2 ):
  with np.errstate(divide='ignore', invalid='ignore'):
    final=np.true_divide(arr1,arr2)
    final[ ~ np.isfinite(final)] = 0  
  return final

In [0]:
!pip install cvxpy

In [0]:
from cvxpy import *
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.utils import check_array 
from sklearn.metrics import mean_absolute_error
!pip install fancyimpute

In [0]:
from cvxpy import installed_solvers
cvxpy.installed_solvers()

['ECOS', 'ECOS_BB', 'SCS', 'OSQP']

In [0]:


"""
  Solve using a nuclear norm approach, using CVXPY.
  [ Candes and Recht, 2009 ]
  Parameters:
  -----------
  Y : m x n array
    matrix we want to complete        
  R : m x n array
    matrix with entries zero (if missing) or one (if present)
  lambda : float
    hyper-parameter controlling trade-off between nuclear norm and square loss
  Returns:
  --------
  X: m x n array
    completed matrix 
 return X.value
"""
# !pip install superscs
# from __future__ import absolute_import, print_function, division



def Nuclear_Norm(Y,R,lmbda):
  X = Variable(Y.shape)
  print("polo")
  obj_fn = Minimize( lmbda * normNuc(X) + sum_squares(multiply(R , X-Y)))
#   print(obj_fn)
  constraints = [X>=0]
  prob = Problem(obj_fn,constraints)
  print("lopppo")
  print(prob)
  prob.solve(verbose = True,solver="SCS") #Splitting Cone Solver
  print("lol")
  print(X.value)
  return X.value

from fancyimpute import NuclearNormMinimization
# import superscs

def NMAE(test,pred):
  tm = mean(test)
  pm = mean(pred)
  den = tm*pm
  nmae = mean_absolute_error(test,pred)/den
  return nmae

kf = KFold(n_splits=5)

for traini,testi in kf.split(dataframe):
  train = dataframe.iloc[traini]
  test = dataframe.iloc[testi]
  mat_train = np.zeros((Num_users,Num_mov))
  mat_test = np.zeros((Num_users,Num_mov))
  
  for i in train.itertuples():
    mat_train[i[1]-1,i[2]-1] = i[3]
  for i in test.itertuples():
    mat_test[i[1]-1,i[2]-1] = i[3]
  print(mat_test.shape,mat_train.shape)
  
  R = np.zeros((Num_users,Num_mov))
  for i in range(len(mat_train)):
    for j in range(len(mat_train[0])):
      if(mat_train[i][j] != 0):
        R[i][j] = 1

 
  
  # matrix completion using convex optimization to find low-rank solution
# that still matches observed values. Slow!
  print(mat_train)
#   mat_train[mat_train==0] = np.nan
  print(mat_train)
#   pred = NuclearNormMinimization().fit_transform(mat_train)
  
  
  pred = Nuclear_Norm(mat_train,R,0.2)
  
  nmae = nmae(mat_test,pred)
  
#   Rt = np.zeros((Num_users,Num_mov))
#   for i in range(len(mat_test)):
#     for j in range(len(mat_test[0])):
#       if(mat_test[i][j] != 0):
             
  
  
  
  print("done")
  
  


(943, 1682) (943, 1682)
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
polo
lopppo
minimize 0.2 * normNuc(var21) + quad_over_lin([[1. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]] * (var21 + -[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]), 1.0)
subject to 0.0 + -var21 <= 0


SystemError: ignored

In [0]:
import numpy

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""


def Nuclear_Norm(Y,R,lmbda):
  X = Variable(*Y.shape)
  obj_fn = Minimize( lmbda * norm(X,"nuc") + sum_squares(mul_elemwise(R , X-Y)))
  
  prob = Problem(obj_fn, [])
  prob.solve(solver=SCS) #Splitting Cone Solver
  return X.value


# def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, lambdaa=0.02):
#     Q = Q.T
#     for step in range(steps):
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
#                     for k in range(K):
#                         P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - lambdaa * P[i][k])
#                         Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - lambdaa * Q[k][j])
#         eR = numpy.dot(P,Q)
#         e = 0
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
#                     for k in range(K):
#                         e = e + (lambdaa/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
#                         e = e + (lambdaa/2) * Nuclear_Norm(Y,R,lambdaa)
#         if e < 0.001:
#             break
#     return P, Q.T

# y r formulate 

###############################################################################

if __name__ == "__main__":
    R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    R = numpy.array(R)

    N = len(R)
    M = len(R[0])
    K = 2

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)

In [0]:
!pip install cvxopt

    100% |████████████████████████████████| 11.6MB 3.6MB/s 


In [0]:
from cvxpy import *
from cvxopt import solvers
# y = M, omega = R
def _solve(M, omega):
    n_1 = M.shape[0]
    n_2 = M.shape[1]
    X_ = Semidef(n_1 + n_2)
    objective = Minimize(trace(X_))

    constraints = [(X_ == X_.T)]  # add symmetric constraint.
    for i, j in omega:
        constr = (X_[i, j + n_1] == M[i, j])
        constraints.append(constr)
    problem = Problem(objective, constraints)
    problem.solve(solver=CVXOPT)

#     print("STATUS       :", problem.status)
    print("OPTIMAL VALUE:", problem.value)

    X0 = X_.value
    # check optimizer's solution is symmetric.
    print("|X0-X0.T|_F  :", np.linalg.norm(np.subtract(X0, X0.T), "fro"))
    return X_.value[:n_1, n_1:]

  
def Nuclear_Norm(Y,R,lmbda):
  X = Variable(Y.shape)
  print("polo")
  obj_fn = Minimize( lmbda * normNuc(X) + sum_squares(multiply(R , X-Y)))
#   print(obj_fn)
  constraints = [X>=0]
  prob = Problem(obj_fn,constraints)
  print("lopppo")
  print(prob)
  prob.solve(verbose = True,solver=CVXOPT) #Splitting Cone Solver
  print("lol")
  print(X.value)
  return X.value


def NMAE(test,pred):
  tm = mean(test)
  pm = mean(pred)
  den = tm*pm
  nmae = mean_absolute_error(test,pred)/den
  return nmae

#################################################

# M training - - true matrix
# r = rank of matrix
# omega - observed entry
# M_
# X - solution to nuclear norm

kf = KFold(n_splits=5)

for traini,testi in kf.split(dataframe):
  train = dataframe.iloc[traini]
  test = dataframe.iloc[testi]
  mat_train = np.zeros((Num_users,Num_mov))
  mat_test = np.zeros((Num_users,Num_mov))
  
  for i in train.itertuples():
    mat_train[i[1]-1,i[2]-1] = i[3]
  for i in test.itertuples():
    mat_test[i[1]-1,i[2]-1] = i[3]
  print(mat_test.shape,mat_train.shape)
  
  R = np.zeros((Num_users,Num_mov))
  for i in range(len(mat_train)):
    for j in range(len(mat_train[0])):
      if(mat_train[i][j] != 0):
        R[i][j] = 1

 
  
  print(mat_train)
#   mat_train[mat_train==0] = np.nan
  print(mat_train)
#   pred = NuclearNormMinimization().fit_transform(mat_train)
  
  
  pred = Nuclear_Norm(mat_train,R,0.2)
  
  nmae = nmae(mat_test,pred)
  
#   Rt = np.zeros((Num_users,Num_mov))
#   for i in range(len(mat_test)):
#     for j in range(len(mat_test[0])):
#       if(mat_test[i][j] != 0):
             
  
  
  
  print("done")
  
  


(943, 1682) (943, 1682)
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
polo
lopppo
minimize 0.2 * normNuc(var58) + quad_over_lin([[1. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]] * (var58 + -[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]), 1.0)
subject to 0.0 + -var58 <= 0


SolverError: ignored

In [0]:

import random
import math
import itertools

import numpy as np
from cvxpy import *


def _solve(M, omega):
    n_1 = M.shape[0]
    n_2 = M.shape[1]
    X = Variable(M.shape)
    print("lalal",X,len(omega),len(omega[0]))
    _get_mask_matrix(n_1,n_2,omega)
#     X_ = Semidef(n_1 + n_2)
    objective = Minimize( 0.02 * normNuc(X) + sum_squares(multiply(omega , X-M)))

    constraints = [(X_ == X_.T)]  # add symmetric constraint.
    for i, j in omega:
        constr = (X_[i, j + n_1] == M[i, j])
        constraints.append(constr)
    problem = Problem(objective, constraints)
    problem.solve(solver=CVXOPT)

    print("STATUS       :", problem.status)
    print("OPTIMAL VALUE:", problem.value)

    X0 = X_.value
    # check optimizer's solution is symmetric.
    print("|X0-X0.T|_F  :", np.linalg.norm(np.subtract(X0, X0.T), "fro"))
    return X_.value[:n_1, n_1:]


def _generate_omega(n_1, n_2, m):
    return random.sample([(i, j) for i in range(n_1) for j in range(n_2)], m)


def _get_mask_matrix(n_1, n_2, omega):
    """
    If we observed entry (i, j) of matrix M, the entry of mask matrix is 1,
    Otherwise 0.
    """
    mask = np.zeros((n_1, n_2), dtype=np.int8)
    for i, j in omega:
        mask[i, j] = 1
    return mask


def _get_abs_max_from_matrix(M):
    return np.max(np.absolute(M))


def main(n_1, n_2, r, m):
    print("#row of M    :", n_1)
    print("#column of M :", n_2)
    print("#sample      :", m)
    L = np.random.randn(n_1, r)
    R = np.random.randn(n_2, r)
    M = np.dot(L, R.T)
    M_abs_max = _get_abs_max_from_matrix(M)
    # print("RANK of M    :", np.linalg.matrix_rank(M))
    print("|M|_*        :", np.linalg.norm(M, "nuc"))
    M_rank = np.linalg.matrix_rank(M)
    print("RANK of M    :", M_rank)
    U, S, V_T = np.linalg.svd(M, full_matrices=False)
    print(S)

    omega = _generate_omega(n_1, n_2, m)
    mask = _get_mask_matrix(n_1, n_2, omega)

    # M_ is for training data removing the data.
    # This block should not affect the result.
    # Just for defensive programming.
    # M_ = np.random.uniform(-M_abs_max, M_abs_max, M.shape)
    M_ = M.copy()
    np.place(M_, 1 - mask, M_abs_max * M_abs_max)
    X = _solve(M_, omega)
    X_rank = np.linalg.matrix_rank(X)
    print("RANK of X    :", X_rank)
    print("|X|_*        :", np.linalg.norm(X, "nuc"))
    U, S, V_T = np.linalg.svd(X, full_matrices=False)
    print(S)

    E = np.subtract(M, X)
    E_train = E.copy()
    np.place(E_train, 1 - mask, 0)
    print('TRAIN RMSE   :', np.linalg.norm(E_train, "fro") / m)
    E_test = E.copy()
    np.place(E_test, mask, 0)
    print('TEST  RMSE   :', np.linalg.norm(E_test, "fro") / (n_1 * n_2 - m))

    frobenius_norm_ratio = (
        np.linalg.norm(X - M, "fro") / np.linalg.norm(M, "fro"))
    print("|X-M|_F/|M|_F:", frobenius_norm_ratio)  # the metric of paper.


if __name__ == '__main__':
    n = 20
    p = 0.8
    main(n, n, r=2, m=int(n * n * p))

#row of M    : 20
#column of M : 20
#sample      : 320
|M|_*        : 36.82971978810182
RANK of M    : 2
[1.95775918e+01 1.72521279e+01 1.60263931e-15 1.32648777e-15
 1.02265642e-15 8.95381206e-16 6.50428753e-16 4.74856834e-16
 4.51279986e-16 3.43303679e-16 3.37684365e-16 2.57078250e-16
 2.29233158e-16 1.42891264e-16 1.01755255e-16 5.62449041e-17
 4.91068713e-17 2.98441848e-17 2.15980360e-17 9.64238275e-18]
lalal var64 320 2


ValueError: ignored